In [1]:
import dill
with open(r"C:\Users\212392510\Desktop\共有用\02_D_EF\20170317_CF LOSS\MBD20170404.dill","rb") as f:
    MBD = dill.load(f)

In [ ]:
CFENT = pd.read_csv(r"C:\Users\212392510\Desktop\共有用\02_D_EF\20170317_CF LOSS\ACC-DCG_LIST_CF_2017-04-05-05-52-39.zip",compression="zip",
                    header=0,sep='\t',encoding="cp932")

In [8]:
mxw.viewOnXL_new(CFENT.sample(500))

管理債権（メールから取得）

In [ ]:
import win32com.client
import re

obj = win32com.client.Dispatch("Outlook.Application")
ns = obj.GetNamespace("MAPI")
folder = ns.GetDefaultFolder(6)

fold = folder.Folders(r"管理債権")

cnt = 0
data = pd.DataFrame()
for i in fold.Items:
    data = pd.concat([data,pd.DataFrame({0:{'SUB':i.Subject,'Date':str(i.SentOn)}}).T],axis=0,ignore_index=True)

data.Date = pd.to_datetime(data.Date)
data.loc[:,'SUM_CD'] = data.SUB.str.extract('[\(|（](?P<hit>[A-Z0-9]+?)[\)|）]')

In [ ]:
data.sort_values(["Date"],ascending=False)

デフォルトデータ

In [11]:
sqlstr = "SELECT A.deal_id," + \
"        A.transaction_id as trnsA,B.transaction_id as trnsB," + \
"        A.import_time as impdA,B.import_time as impdB," + \
"        A.exposure_amount,A.days_past_due_cnt," + \
"        B.p_kubun,B.sor_party_id,B.party_role " + \
"FROM DBO.TRANSACTION_FACILITY A " + \
"FULL OUTER JOIN DBO.TRANSACTION_PARTY B " + \
"ON A.TRANSACTION_ID = B.TRANSACTION_ID AND A.IMPORT_TIME = B.IMPORT_TIME " + \
"where B.p_kubun = 'P1' AND days_past_due_cnt > 0"

IRISD = pd.read_sql(sqlstr,cns.JEDW)
DLQD = IRISD.query('days_past_due_cnt >= 30 & party_role == "BORROWER"')\
         .sort_values(['deal_id','days_past_due_cnt','impdA'])\
         .groupby(['deal_id']).nth(-1).loc[:,['impdA','days_past_due_cnt']].reset_index()

In [12]:
# TRANSACTION_IDの変換用関数
def creanTRANNO(x):
    if len(x) in [11,12] :
        return(x[0:-3])
    else:
        if x[-5:] == "00101":
            return(x[0:-5])
        else:
            return(x)

JDEFS = pd.read_sql("SELECT * FROM DEFAULT_TRANSACTION",cns.JEDW)
JDEFS.loc[:,'DSDATE'] = JDEFS.DEFAULT_EPISODE_START_DATE.apply(lambda x:pd.datetime.strptime(x,'%m/%d/%Y') if x != "" else "")
DEFS = pd.read_sql("SELECT * FROM DEFAULT_TRANSACTION",cns.EDWADMIN)
DEFS.loc[:,'DSDATE'] =  pd.to_datetime(DEFS.DEFAULT_EPISODE_START_DATE)
DEFDATA = pd.concat([DEFS,JDEFS]).assign(TRANSAID = lambda x:x.DEFAULT_TRANSACTION_ID.apply(creanTRANNO))\
                     .filter(regex="TRANSA|DSDATE|BUS|DATAFROM")\
                     .sort_values(['DEFAULT_TRANSACTION_ID','DSDATE']).groupby(['DEFAULT_TRANSACTION_ID']).nth([0]).reset_index()

In [13]:
CFWO = pd.read_sql("SELECT * FROM CFWO",cns.SMERISK)
CFREC = pd.read_sql("SELECT * FROM CFREC",cns.SMERISK)
CFWOREC = pd.merge(left=CFWO.groupby(['cont_no']).WO_Amount.sum().reset_index(),
                                  right=CFREC.groupby(['cont_no']).Recovery_Amount.sum().reset_index(),
                                  left_on=['cont_no'],right_on=['cont_no'],how='outer',indicator="WORECMG")

In [14]:
MBDD = MBD.pipe(lambda x:pd.merge(left=x,right=DLQD,left_on=['KEIYAKUNO'],right_on=['deal_id'],how="left"))\
                    .assign(WOFLG = lambda x:x.KEIYAKUNO.isin(CFWO.cont_no))\
                    .assign(KANRI = lambda x:x.SUM_CD.isin(list(data.SUM_CD.dropna().unique()) + ["SEWKTT"]))\
                    .pipe(lambda x:pd.merge(left=x,right=CFWOREC,left_on=['KEIYAKUNO'],right_on=['cont_no'],how='left'))\
                    .pipe(lambda x:pd.merge(left=x,right=DEFDATA,left_on=['KEIYAKUNO'],right_on=['TRANSAID'],how='left'))\
                    .assign(RDEF = lambda x:((x.WOFLG == True)|(x.KANRI==True)))\
                    .assign(RDEF2 = lambda x:((x.WOFLG == True)|(x.KANRI==True)|(~x.DSDATE.isnull())))

In [15]:
# 年度データの作成
MBDD.loc[~MBDD.EO_CONTRACTDATE.isnull(),'CNTYEAR'] = \
MBDD.loc[~MBDD.EO_CONTRACTDATE.isnull(),:].EO_CONTRACTDATE.dt.date - relativedelta(months=3)+ relativedelta(month=4,day=1)

In [16]:
# ACDスコア
labels2 = ["{0} - {1}".format(i*10,i*10 + 9) for i in range(10,21)]
labels3 = ["{0} - {1}".format(i*5,i*5 + 4) for i in range(20,43)]
MBDD.loc[~MBDD.scoren.isnull(),'score_cat'] =MBDD.pipe(lambda x:pd.cut(x.scoren.dropna(),[i*10 for i in range(10,22)],labels=labels2,right=False))
MBDD.loc[~MBDD.scoren.isnull(),'score_cat2'] =MBDD.pipe(lambda x:pd.cut(x.scoren.dropna(),[i*5 for i in range(20,44)],labels=labels3,right=False))

顧客ベースデータ

In [20]:
MBDDC = MBDD.sort_values(['SUM_CD','CNTYEAR','RDEF']).groupby(['SUM_CD','CNTYEAR']).nth(-1).reset_index()\
               .pipe(lambda x:x.loc[~x.CREATION_DATE_FIRST.isnull(),:])\
               .assign(YEAR = lambda x:x.CREATION_DATE_FIRST.dt.year)

In [41]:
SCORE_SUM = pd.pivot_table(MBDDC.pipe(lambda x:x.loc[(~x.KEIYAKUNO.isnull()) & 
                                 ((x.EO_ACD_APPROVAL_KBN.isin(['1','2']))|(x.EO_CA_ACDRINGI == 'Y')),:]),index=['score_cat'],\
                      columns=['CNTYEAR','RDEF'],values="APL_NO",aggfunc="count",fill_value=0,margins=True)

In [21]:
labels_sl = ["{0} - {1}".format(i*0.5,i*0.5 + 0.5) for i in range(0,19)]
MBDDC = MBDDC.pipe(lambda x:pd.concat([x,x.SALES_APL.str.split(",",expand=True).add_prefix("SALES_")],axis=1))\
                          .assign(LSALE = lambda x:x.SALES_0.astype('float64').apply(lambda y:np.log10(y) if np.float64(y) > 0 else None))
MBDDC.loc[~MBDDC.LSALE.isnull(),'LSALEC'] = pd.cut(MBDDC.LSALE.dropna(),[i*0.5 for i in range(0,20)],labels=labels_sl,right=False)

In [22]:
MBDDC = MBDDC.assign(
    RATE = lambda x:x.apply(lambda y:np.float64(y.EO_CA_OEC)/(np.float64(y.SALES_0)*1000000) if np.float64(y.SALES_0) > 0 else None,axis=1))

In [45]:
import seaborn as sns
%matplotlib inline
fig = plt.scatter(x=MBDDC.LSALE,y=MBDDC.scoren,c=MBDDC.RDEF)
fig.figure.set_size_inches(8,8)

In [ ]:
MBDDC.SALES_0

In [ ]:
fig = plt.scatter(x=MBDDC.RATE,y=MBDDC.scoren,c=MBDDC.RDEF,marker=".")
fig.figure.set_size_inches(12,12)

In [162]:
mdata = MBDDC.filter(regex="SALES_0|TDB_SCORE_APL|scoren|^RATE$|EO_CA_OEC")\
                        #.pipe(lambda x:pd.concat([x,pd.get_dummies(MBDDC.EO_CA_MACHINETYPE)],axis=1))

In [47]:
import xgboost as xgb
import scipy as sp
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score

RandomizedSearchでパラメータを調整

In [167]:
params = {'learning_rate':sp.stats.uniform(0.0,0.6),
                 'max_depth':sp.stats.randint(1,6),
                 'subsample':sp.stats.uniform(0.7,0.3),
                 'n_estimators':sp.stats.randint(1,4)}

weight = MBDDC.RDEF.apply(lambda x: 50 if x else 1)
clf2 = xgb.XGBClassifier(reg_alpha=1.3,objective='binary:logistic')
clf2_cv = RandomizedSearchCV(clf2,params,fit_params={'sample_weight':weight},cv=5,n_iter=100,scoring="neg_log_loss")
mod2 = clf2_cv.fit(mdata.as_matrix(),MBDDC.RDEF.apply(lambda x:1 if x else 0))
pred2 = clf2_cv.predict(mdata.as_matrix())
print(clf2_cv.best_params_)

{'max_depth': 5, 'n_estimators': 3, 'subsample': 0.76586457665924734, 'learning_rate': 0.575486281603242}


In [168]:
print(confusion_matrix(pred2,MBDDC.RDEF.apply(lambda x:1 if x else 0)))
print(accuracy_score(pred2,MBDDC.RDEF.apply(lambda x:1 if x else 0)))

[[3446    1]
 [  66   17]]
0.981019830028


In [95]:
mod2.best_estimator_

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.4838948438142765, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=3,
       nthread=-1, objective='binary:logistic', reg_alpha=1.3,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.85213689105640589)

In [ ]:
import seaborn as sns
%matplotlib inline
fig = xgb.plot_importance(mod2.best_estimator_)
fig.figure.set_size_inches(12,8)

In [ ]:
fig = xgb.plot_tree(mod2.best_estimator_,num_trees=2)
fig.figure.set_size_inches(25,20)

In [103]:
MBDDC.loc[:,'PRED_RS'] = pred2

In [106]:
mxw.viewOnXL_new(
MBDDC.query("PRED_RS == 1").filter(regex="CORPKANJI|SALES_|LSALE|RDEF|^RATE$|TDB_SCORE|score|PRED|EO_CA_OEC|MACHINE|deal_final_decision"))

In [ ]:
pd.DataFrame({"feature":pd.Series(list(mdata.columns)),"imp":pd.Series(mod2.best_estimator_.feature_importances_)}).sort_values(['imp'],ascending=False)

手でパラメータ調整

In [335]:
import xgboost as xgb
clf = xgb.XGBClassifier(learning_rate=0.01,max_depth=6,n_estimators=3,subsample=0.9,reg_alpha=1.3,objective="binary:logistic")

In [336]:
weight = MBDDC.RDEF.apply(lambda x: 33 if x else 1)
mod = clf.fit(mdata.as_matrix(),MBDDC.RDEF.apply(lambda x:1 if x else 0),sample_weight=weight)
bst = mod.booster()
pred = clf.predict(mdata.as_matrix())
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(pred,MBDDC.RDEF.apply(lambda x:1 if x else 0)))
print(accuracy_score(pred,MBDDC.RDEF.apply(lambda x:1 if x else 0)))
MBDDC.loc[:,'PRED'] = pred

[[3455    1]
 [  57   17]]
0.983569405099


フィットさせるためにCVをやる場合

In [274]:
params = {"leaning_rate":0.2,"max_depth":6,"n_estimators":3,"reg_alpha":0.5,"objective":"binary:logistic"}
dtrain = xgb.DMatrix(mdata.as_matrix(),label=MBDDC.RDEF.apply(lambda x:1 if x else 0),feature_names=mdata.columns)
cv = xgb.cv(params,dtrain,num_boost_round=3,nfold=30)
cv.loc[:,['train-error-mean','train-error-std']].plot()
bst2 = xgb.train(params,dtrain,num_boost_round=100)

------------------------------------

In [ ]:
pd.pivot_table(MBDDC,index='PRED',columns=None,values=['RATE'],aggfunc="mean",fill_value=0)

In [ ]:
pd.pivot_table(MBDDC,index='RDEF',columns=None,values=['RATE'],aggfunc="mean",fill_value=0)

In [164]:
mxw.viewOnXL_new(MBDDC.query('RDEF==True'))

C:\Users\212392510\AppData\Local\Continuum\Anaconda3\lib\site-packages\xlwings\_xlwindows.py:1024: FutureWarning:

to_datetime is deprecated. Use self.to_pydatetime()



In [234]:
mxw.viewOnXL_new(MBDDC.query('(RATE > 0.1) & (deal_final_decision_result != 3) | (RDEF == True)')\
                 .filter(regex="CORPKANJI|SALES_|LSALE|RDEF|^RATE$|TDB_SCORE|score|PRED|EO_CA_OEC|MACHINE|deal_final_decision"))

In [155]:
def plotGrid(data,tgtval,x_size,y_size,**kwargs):
    g = sns.FacetGrid(data,**kwargs)
    g.map(sns.distplot,tgtval)
    g.fig.set_size_inches(x_size,y_size)

In [ ]:
plotGrid(MBDDC,'RATE',12,12,row='PRED')

In [ ]:
fig = xgb.plot_tree(mod,num_trees=0)
fig.figure.set_size_inches(25,20)

集計して建機の比率を見てみる

In [ ]:
pd.pivot_table(MBDDC.pipe(lambda x:x.loc[x.EO_ACD_APPROVAL_KBN.isin(["1","2"]),:]),
               index="EO_CA_MACHINETYPE",columns=["CNTYEAR","RDEF"],values="scoren",aggfunc="count",margins="All",fill_value=0)\
.assign(pct=lambda x:x.iloc[:,1]/x.All)

In [269]:
1-(1-0.0075)**(1/2)

0.0037570577414361983

In [ ]:
mxw.viewOnXL_new(MBDDC.query("PRED==1")

In [ ]:
mdata.sample(10)

In [ ]:
pd.DataFrame(bst.get_dump(),columns=["TREE"]).TREE.str.replace("\n","-")

In [ ]:
xgb.plot_importance(mod)

In [161]:
mdata2 = MBDDC.filter(regex="SALES_0|TDB_SCORE_APL|scoren|^RATE$|EO_CA_OEC").dropna()

In [ ]:
fig = xgb.plot_tree(mod,num_trees=0)
fig.figure.set_size_inches(25,20)

In [ ]:
bst.get_dump()[0]

In [ ]:
MBDD.query("RDEF == True").pipe(lambda x:x.loc[x.SUM_CD.isin(["G12957","644342"])])

In [ ]:
MBDD.pipe(lambda x:x.loc[x.CORPKANJI_NM_APL.str.contains("吉田工作所").fillna(False),:])